1. **Assume that we have some data $x_1, \ldots, x_n \in \mathbb{R}$. Our goal is to find a constant $b$ such that $\sum_i (x_i - b)^2$ is minimized.**
    1. Find an analytic solution for the optimal value of $b$.
    1. How does this problem and its solution relate to the normal distribution?
    1. What if we change the loss from $\sum_i (x_i - b)^2$ to $\sum_i |x_i-b|$? Can you find the optimal solution for $b$?

A.
$$\sum_i (x_i-b)^2 = \sum_i x_i^2 - 2b\sum_i x_i + nb^2=n(b-\frac{1}{n}\sum_i x_i)^2+\sum_i x_i^2-\frac{(\sum_i x_i)^2}{n}$$
Let $b = \frac{1}{n}\sum_i x_i$, the function gets the minimum value.

B.
Assume that $X = b + \epsilon$ where the noise 𝜖 is normally distributed, if we want to estimate b, then we will minimize $\sum_i (x_i - b)^2$ and the solution is the mean of all samples.

C.
From the [link](https://math.stackexchange.com/questions/113270/the-median-minimizes-the-sum-of-absolute-deviations-the-ell-1-norm)

![3_1_1](material/3_1_1.png)

2. **Prove that the affine functions that can be expressed by $\mathbf{x}^\top \mathbf{w} + b$ are equivalent to linear functions on $(\mathbf{x}, 1)$.**

$$\mathbf{x}^T\mathbf{W}+b=x_1*w_1 + ... + x_n * w_n + 1 * b = (\mathbf{x},1)^T(\mathbf{w},b)$$

3. **Assume that you want to find quadratic functions of $\mathbf{x}$, i.e., $f(\mathbf{x}) = b + \sum_i w_i x_i + \sum_{j \leq i} w_{ij} x_{i} x_{j}$. How would you formulate this in a deep network?**

In [4]:
import torch
import torch.nn as nn

def mul(x):
    n = x.size(0)
    res = torch.zeros(int(n * (n + 1) / 2))
    index = 0
    for i in range(0, n):
        for j in range(i, n):
            res[index] = x[i] * x[j]
            index += 1
    return res

class Model(nn.Module):
    def __init__(self, n): 
        super(Model,self).__init__()
        self.fc1 = nn.Linear(n, 1)
        self.fc2 = nn.Linear(int(n * (n + 1) / 2), 1)
    def forward(self, x):
        y1 = self.fc1(x)
        y2 =mul(x)
        y2 = self.fc2(y2)
        y = y1 + y2
        return y

model = Model(10)
x = torch.randn(10)
y = model(x)

4. **Recall that one of the conditions for the linear regression problem to be solvable was that the design matrix $\mathbf{X}^\top \mathbf{X}$ has full rank.**
    1. What happens if this is not the case?
    1. How could you fix it? What happens if you add a small amount of coordinate-wise independent Gaussian noise to all entries of $\mathbf{X}$?
    1. What is the expected value of the design matrix $\mathbf{X}^\top \mathbf{X}$ in this case?
    1. What happens with stochastic gradient descent when $\mathbf{X}^\top \mathbf{X}$ does not have full rank?

<font color = red>(uncertain)</font>

A. We can not use the $(X^TX)^{-1}$ in the solution and there will be more than one optimal solution for (w,b)

B. We can drop out some dependent dimensions and reduce the number of dimensions of both x and w, called reparameterization. If we add noise to X, the matrix $X^TX$ will have full rank.

C. 
$$E((X+\epsilon)^T(X+\epsilon))=E(X^TX)+E(X^T\epsilon)+E(\epsilon^TX)+E(\epsilon^T\epsilon)=E(X^TX)$$

D. Stochastic gradient descent (SGD) can still be used when (\mathbf{X}^\top \mathbf{X}) does not have full rank, but it may not converge to a unique solution. This is because SGD does not rely on the invertibility of (\mathbf{X}^\top \mathbf{X}), but instead iteratively updates the regression coefficients based on a randomly selected subset of the data. However, the presence of linearly dependent columns in (\mathbf{X}) can cause the loss surface to have multiple minima, which means that SGD may converge to different solutions depending on the initial values of the regression coefficients.

5. **Assume that the noise model governing the additive noise $\epsilon$ is the exponential distribution. That is, $p(\epsilon) = \frac{1}{2} \exp(-|\epsilon|)$.**
    1. Write out the negative log-likelihood of the data under the model $-\log P(\mathbf y \mid \mathbf X)$.
    1. Can you find a closed form solution?
    1. Suggest a minibatch stochastic gradient descent algorithm to solve this problem. What could possibly go wrong (hint: what happens near the stationary point as we keep on updating the parameters)? Can you fix this?

A.
$$P(\mathbf{y}|\mathbf{X})=P(\epsilon=y-Xw)=\prod \frac{1}{2}exp(-|y_i-x_i^Tw|)$$
$$L = -logP(\mathbf{y}|\mathbf{X})=Nlog2+\sum_i \|y_i-x_i^Tw\|$$

B.
$$\bigtriangledown_w(L)=-\sum sign(y_i-xi^Tw)x_i=0$$
Can't find the closed form solution.

C.

The gradient is therefore not a smooth function and has jumps where $sign(y_i-xi^Tw)$ changes its sign. So the function will be hard to converge. We can use a larger batchsize.

6. **Assume that we want to design a neural network with two layers by composing two linear layers. That is, the output of the first layer becomes the input of the second layer. Why would such a naive composition not work?**

The composition of two linear layers in a neural network essentially results in one linear layer. This is due to the property of linearity: the composition of two linear functions is another linear function.

7. **What happens if you want to use regression for realistic price estimation of houses or stock prices?**
    1. Show that the additive Gaussian noise assumption is not appropriate. Hint: can we have negative prices? What about fluctuations?
    1. Why would regression to the logarithm of the price be much better, i.e., $y = \log \textrm{price}$?
    1. What do you need to worry about when dealing with pennystock, i.e., stock with very low prices? Hint: can you trade at all possible prices? Why is this a bigger problem for cheap stock? For more information review the celebrated Black--Scholes model for option pricing

<font color = green>(from the discussion)</font>

A. Prices can't be negative but the gaussian noise allows for negative values. Stock price fluctuations are related to its previous history.

B. Now the range of y is the entire real number domain.

C. If the price takes a very penny value, the logarithm of it will go to a very large negative number, that makes the value changes intensely. The prices of these stocks may not be able to change smoothly due to the minimum tick size, or the smallest increment by which the price can change. This can lead to a discontinuous distribution of price changes, which is not well modeled by a Gaussian distribution.

8. **Suppose we want to use regression to estimate the *number* of apples sold in a grocery store.**
    1. What are the problems with a Gaussian additive noise model? Hint: you are selling apples, not oil.
    1. The [Poisson distribution](https://en.wikipedia.org/wiki/Poisson_distribution) captures distributions over counts. It is given by $p(k \mid \lambda) = \lambda^k e^{-\lambda}/k!$. Here $\lambda$ is the rate function and $k$ is the number of events you see. Prove that $\lambda$ is the expected value of counts $k$.
    1. Design a loss function associated with the Poisson distribution.
    1. Design a loss function for estimating $\log \lambda$ instead.

A.
The number of apple is an integer, so it is discrete. But the Gaussian additive noise is continuous.

B.

$$E(k)=\sum_{k=1}^\infty k \frac{\lambda^k e^{-\lambda}}{k!}=\lambda e^{-\lambda}\sum_{k=1}^\infty \frac{\lambda^{k-1}}{(k-1)!}=\lambda e^{-\lambda}\sum_{k=0}^\infty \frac{\lambda^{k}}{(k)!}=\lambda e^{-\lambda} e^{\lambda}=\lambda$$

C.
$$-log(P(K|\lambda))=-\sum_i (k_ilog\lambda -\lambda - log(k_i!))= n\lambda - log\lambda \sum_i k_i + \sum_i (log(k_i!))$$
The loss function for $\lambda$:
$$L(\lambda)=n\lambda - log\lambda \sum_i k_i$$

D. Let $t = log\lambda$
$$L(t)=ne^t - t \sum_i k_i$$